In [ ]:
import pandas as pd
import numpy as np
import glob
import os

def euclidean_distance(p1, p2):
    return np.sqrt((p1[0] - p2[0]) ** 2 + (p1[1] - p2[1]) ** 2)

def match_colors(csv_file, txt_file, output_file):
    # CSVを読み込む
    df_csv = pd.read_csv(csv_file)
    frame_col = df_csv.columns[0]  # frame_numberのカラム名
    
    # 各座標と色の対応を作成
    color_map = {}
    for col in df_csv.columns[1:]:
        if col.startswith("x_"):
            color = col.split('_')[-1]  # 色名のみ取得
            y_col = f"y_{col[2:]}"
            color_map[color] = df_csv[[frame_col, col, y_col]].rename(columns={col: 'x', y_col: 'y'})
    
    # TXTを読み込む
    df_txt = pd.read_csv(txt_file, header=None, names=['frame', 'id', 'x', 'y'])
    df_txt['color'] = ''  # 新しいカラムを追加
    
    # 各フレームごとに処理
    for frame in df_txt['frame'].unique():
        txt_subset = df_txt[df_txt['frame'] == frame]
        csv_subset = {k: v[v[frame_col] == frame] for k, v in color_map.items()}
        
        for i, row in txt_subset.iterrows():
            min_dist = float('inf')
            best_match = None
            
            for color, df in csv_subset.items():
                if not df.empty:
                    dist = euclidean_distance((row['x'], row['y']), (df.iloc[0]['x'], df.iloc[0]['y']))
                    if dist < min_dist:
                        min_dist = dist
                        best_match = color
            
            if best_match:
                df_txt.at[i, 'color'] = best_match[2:]  # 色名のみを設定
    
    # 出力
    df_txt.to_csv(output_file, index=False, header=False)

# ファイル一覧を取得
csv_files = sorted(glob.glob("../../ground_truth/Indoor/tracking_data_with_color/*.csv"))
txt_files_pre = sorted(glob.glob("../../ground_truth/Indoor/transformed_MOT/*_pre.txt"))
txt_files_post = sorted(glob.glob("../../ground_truth/Indoor/transformed_MOT/*_post.txt"))

txt_files = txt_files_pre + txt_files_post  # preとpostの両方を処理

# ファイル名の対応を取る
for csv_file, txt_file in zip(csv_files, txt_files):
    output_file = txt_file.replace(".txt", "_color.txt")
    match_colors(csv_file, txt_file, output_file)
    print(f"Processed {output_file}")

Processed ground_truth/Indoor/transformed_MOT/basket_S1T1_pre_color.txt
Processed ground_truth/Indoor/transformed_MOT/basket_S1T2_pre_color.txt
Processed ground_truth/Indoor/transformed_MOT/basket_S1T3_pre_color.txt
Processed ground_truth/Indoor/transformed_MOT/basket_S1T4_pre_color.txt
Processed ground_truth/Indoor/transformed_MOT/basket_S1T5_pre_color.txt
Processed ground_truth/Indoor/transformed_MOT/basket_S1T6_pre_color.txt
Processed ground_truth/Indoor/transformed_MOT/basket_S1T7_pre_color.txt
Processed ground_truth/Indoor/transformed_MOT/basket_S2T1_pre_color.txt
Processed ground_truth/Indoor/transformed_MOT/basket_S2T2_pre_color.txt
Processed ground_truth/Indoor/transformed_MOT/basket_S2T3_pre_color.txt
Processed ground_truth/Indoor/transformed_MOT/basket_S2T4_pre_color.txt
Processed ground_truth/Indoor/transformed_MOT/basket_S2T5_pre_color.txt
Processed ground_truth/Indoor/transformed_MOT/basket_S2T6_pre_color.txt
Processed ground_truth/Indoor/transformed_MOT/basket_S2T7_pre_co

In [2]:
import os
import pandas as pd

# CSV を読み込む
csv_path = "initial_location.csv"
df = pd.read_csv(csv_path)

# 入力ディレクトリと出力ディレクトリ
input_dir = "ground_truth/Indoor/transformed_MOT/"
output_dir = "ground_truth/Indoor/transformed_MOT_fixed/"

# 出力ディレクトリを作成（存在しない場合）
os.makedirs(output_dir, exist_ok=True)

# ファイル一覧を取得
txt_files = [f for f in os.listdir(input_dir) if f.startswith("basket_") and f.endswith("_color.txt")]

# 置換対象の色リスト
colors = ["red", "blue", "pink", "black", "orange", "yellow"]

# 各 TXT ファイルを処理
for txt_file in txt_files:
    input_path = os.path.join(input_dir, txt_file)
    output_path = os.path.join(output_dir, txt_file)

    # ファイル名からキーを取得 (例: "basket_S1T1_pre_color.txt" → "S1T1")
    key = txt_file.split("_")[1]  # "S1T1"

    # CSV に対応する行を取得
    if key in df["file"].values:
        mapping = df[df["file"] == key].iloc[0, 1:].to_dict()  # {red: O_top, blue: O_right, ...}

        # TXTファイルを読み込む
        with open(input_path, "r") as f:
            lines = f.readlines()

        # 置換処理
        new_lines = []
        for line in lines:
            for color in colors:
                if color in mapping:
                    line = line.replace(color, mapping[color])
            new_lines.append(line)

        # 変更後の内容を新ディレクトリに保存
        with open(output_path, "w") as f:
            f.writelines(new_lines)

print(f"すべてのTXTファイルを {output_dir} に保存しました。")


すべてのTXTファイルを ground_truth/Indoor/transformed_MOT_fixed/ に保存しました。


In [3]:
import os

# 対象ディレクトリ
dir_path = "ground_truth/Indoor/transformed_MOT_fixed/"

# ディレクトリ内のファイルを取得
for filename in os.listdir(dir_path):
    if filename.endswith(".txt") and "_color" in filename:
        old_path = os.path.join(dir_path, filename)
        new_filename = filename.replace("_color", "")
        new_path = os.path.join(dir_path, new_filename)

        # ファイル名を変更
        os.rename(old_path, new_path)
        print(f"Renamed: {filename} -> {new_filename}")

print("すべてのファイルのリネームが完了しました。")

Renamed: basket_S1T1_pre_color.txt -> basket_S1T1_pre.txt
Renamed: basket_S1T2_pre_color.txt -> basket_S1T2_pre.txt
Renamed: basket_S1T3_pre_color.txt -> basket_S1T3_pre.txt
Renamed: basket_S1T4_pre_color.txt -> basket_S1T4_pre.txt
Renamed: basket_S1T5_pre_color.txt -> basket_S1T5_pre.txt
Renamed: basket_S1T6_pre_color.txt -> basket_S1T6_pre.txt
Renamed: basket_S1T7_pre_color.txt -> basket_S1T7_pre.txt
Renamed: basket_S2T1_pre_color.txt -> basket_S2T1_pre.txt
Renamed: basket_S2T2_pre_color.txt -> basket_S2T2_pre.txt
Renamed: basket_S2T3_pre_color.txt -> basket_S2T3_pre.txt
Renamed: basket_S2T4_pre_color.txt -> basket_S2T4_pre.txt
Renamed: basket_S2T5_pre_color.txt -> basket_S2T5_pre.txt
Renamed: basket_S2T6_pre_color.txt -> basket_S2T6_pre.txt
Renamed: basket_S2T7_pre_color.txt -> basket_S2T7_pre.txt
Renamed: basket_S3T1_pre_color.txt -> basket_S3T1_pre.txt
Renamed: basket_S3T2_pre_color.txt -> basket_S3T2_pre.txt
Renamed: basket_S3T3_pre_color.txt -> basket_S3T3_pre.txt
Renamed: baske